In [1]:
!pip install -q -U kaggle_environments

In [3]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [4]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [5]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [6]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [7]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

В игре примут участие агенты, придерживающиеся следующих стратегий:  

1. **Всегда Камень**  
2. **Всегда Бумага**  
3. **Всегда Ножницы**  
4. **Циклическая смена**: Камень → Бумага → Ножницы  
5. **Циклическая смена**: Ножницы → Камень → Бумага  
6. **Циклическая смена**: Бумага → Ножницы → Камень  
7. **Повторяющийся шаблон**: Камень → Камень → Ножницы → Ножницы → Бумага → Бумага  
8. **Повторяющийся шаблон**: Ножницы → Ножницы → Бумага → Бумага → Камень → Камень  
9. **Повторяющийся шаблон**: Бумага → Бумага → Камень → Камень → Ножницы → Ножницы  
10. **Повторяющийся шаблон**: Камень → Камень → Камень → Ножницы → Ножницы → Ножницы → Бумага → Бумага → Бумага  
11. **Повторяющийся шаблон**: Ножницы → Ножницы → Ножницы → Бумага → Бумага → Бумага → Камень → Камень → Камень  
12. **Повторяющийся шаблон**: Бумага → Бумага → Бумага → Камень → Камень → Камень → Ножницы → Ножницы → Ножницы  
13. **Камень, затем контр-ход**: первый ход — Камень, все последующие — контр-ходы на предыдущие действия противника.

В коде: 0 - камень, 1 - бумага, 2 - ножницы



In [8]:
%%writefile rock_agent.py
def rock_agent(observation, configuration): #тут и далее observation-информация о текущем состоянии игры(номер текущего хода,  предыдущий ход противника)
#configuration содержит информацию о правилах и настройках игры (все это реализовано ранее)

  #0 - камень, 1 - бумага, 2 - ножницы
  #Агент, всегда играющий "Камень"

  return 0

Overwriting rock_agent.py


In [9]:
%%writefile paper_agent.py
def paper_agent(observation, configuration):

  #Агент, всегда играющий "Бумага"

  return 1

Overwriting paper_agent.py


In [10]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
  #Агент, всегда играющий "Ножницы"

  return 2

Overwriting scissors_agent.py


In [11]:
%%writefile cycle_agent.py
def cycle_agent(observation, configuration):
    #Агент, постоянно ходящий по кругу, начиная с камня

    if observation.step == 0: # observation.step содержит номер текущего хода
        return 0  # Начинаем с камня
    else:
        return (observation.step) % 3 # 0, 1, 2, 0, 1, 2, ...

Overwriting cycle_agent.py


In [12]:
%%writefile scissors_cycle_agent.py
def scissors_cycle_agent(observation, configuration):
    #Агент, постоянно ходящий по кругу, начиная с ножниц

    if observation.step == 0: # observation.step содержит номер текущего хода
        return 2  # Начинаем с ножниц
    else:
        return (observation.step + 2) % 3 # 2, 0, 1, 2, 0, 1, ...

Overwriting scissors_cycle_agent.py


In [13]:
%%writefile paper_cycle_agent.py
def paper_cycle_agent(observation, configuration):
    #Агент, постоянно ходящий по кругу, начиная с бумаги

    if observation.step == 0: # observation.step содержит номер текущего хода
        return 1  # Начинаем с бумаги
    else:
        return (observation.step + 1) % 3 # 1, 2, 0, 1, 2, 0, ...

Overwriting paper_cycle_agent.py


In [14]:
%%writefile pattern_agent_kknnbb.py
def pattern_agent_kknnbb(observation, configuration):
    #Агент, постоянно повторяющий шаблон: два раза камень, два раза ножницы, два раза бумага.

    if observation.step == 0: # observation.step содержит номер текущего хода
        return 0  # Первый ход - камень
    elif observation.step == 1:
        return 0  # Второй ход - камень
    elif observation.step == 2:
        return 2  # Третий ход - ножницы
    elif observation.step == 3:
        return 2 # Четвертый ход - ножницы
    elif observation.step == 4:
        return 1 # Пятый ход - бумага
    elif observation.step == 5:
        return 1 # Шестой ход - бумага
    else:
        return (observation.step % 6) // 2 # Повторяем цикл

Overwriting pattern_agent_kknnbb.py


In [15]:
%%writefile repeating_pattern_agent_nnbbkk.py
def repeating_pattern_agent_nnbbkk(observation, configuration):
    #Агент, постоянно повторяющий шаблон: два раза ножницы, два раза бумага, два раза камень.

    step = observation.step  # Получаем текущий номер хода

    # Определяем ход агента в зависимости от номера хода:
    if step % 6 == 0:  # Если остаток от деления на 6 равен 0 (ходы 0, 6, 12...)
        return 2  # Играем ножницы (2)
    elif step % 6 == 1:  # Если остаток от деления на 6 равен 1 (ходы 1, 7, 13...)
        return 2  # Играем ножницы (2)
    elif step % 6 == 2:  # Если остаток от деления на 6 равен 2 (ходы 2, 8, 14...)
        return 1  # Играем бумагу (1)
    elif step % 6 == 3:  # Если остаток от деления на 6 равен 3 (ходы 3, 9, 15...)
        return 1  # Играем бумагу (1)
    elif step % 6 == 4:  # Если остаток от деления на 6 равен 4 (ходы 4, 10, 16...)
        return 0  # Играем камень (0)
    elif step % 6 == 5:  # Если остаток от деления на 6 равен 5 (ходы 5, 11, 17...)
        return 0  # Играем камень (0)

Overwriting repeating_pattern_agent_nnbbkk.py


In [16]:
%%writefile repeating_pattern_agent_bbkknn.py
def repeating_pattern_agent_bbkknn(observation, configuration):
    if observation.step == 0:
        return 1  # Бумага
    elif observation.step == 1:
        return 1  # Бумага
    elif observation.step == 2:
        return 0  # Камень
    elif observation.step == 3:
        return 0  # Камень
    elif observation.step == 4:
        return 2  # Ножницы
    elif observation.step == 5:
        return 2  # Ножницы
    else:
        if observation.step % 6 == 0 or observation.step % 6 == 1:
            return 1 # Бумага
        elif observation.step % 6 == 2 or observation.step % 6 == 3:
            return 0 # Камень
        else:
            return 2 # Ножницы

Overwriting repeating_pattern_agent_bbkknn.py


In [17]:
%%writefile repeating_pattern_agent_kkknnnbbb.py
def repeating_pattern_agent_kkknnnbbb(observation, configuration):
    if observation.step == 0:
        return 0  # Камень
    elif observation.step == 1:
        return 0  # Камень
    elif observation.step == 2:
        return 0 # Камень
    elif observation.step == 3:
        return 2  # Ножницы
    elif observation.step == 4:
        return 2  # Ножницы
    elif observation.step == 5:
        return 2 # Ножницы
    elif observation.step == 6:
        return 1  # Бумага
    elif observation.step == 7:
        return 1  # Бумага
    elif observation.step == 8:
        return 1 # Бумага
    else:
         if 0 <= observation.step % 9 < 3:
             return 0
         elif 3 <= observation.step % 9 < 6:
             return 2
         else:
            return 1

Overwriting repeating_pattern_agent_kkknnnbbb.py


In [18]:
%%writefile repeating_pattern_agent_nnnbbbkkk.py
def repeating_pattern_agent_nnnbbbkkk(observation, configuration):
    if observation.step == 0:
        return 2  # Ножницы
    elif observation.step == 1:
        return 2  # Ножницы
    elif observation.step == 2:
        return 2  # Ножницы
    elif observation.step == 3:
        return 1  # Бумага
    elif observation.step == 4:
        return 1  # Бумага
    elif observation.step == 5:
        return 1  # Бумага
    elif observation.step == 6:
        return 0  # Камень
    elif observation.step == 7:
        return 0  # Камень
    elif observation.step == 8:
        return 0  # Камень
    else:
         if 0 <= observation.step % 9 < 3:
             return 2
         elif 3 <= observation.step % 9 < 6:
             return 1
         else:
            return 0

Overwriting repeating_pattern_agent_nnnbbbkkk.py


In [19]:
%%writefile repeating_pattern_agent_bbbkkknnn.py
def repeating_pattern_agent_bbbkkknnn(observation, configuration):
    if observation.step == 0:
        return 1  # Бумага
    elif observation.step == 1:
        return 1  # Бумага
    elif observation.step == 2:
        return 1  # Бумага
    elif observation.step == 3:
        return 0  # Камень
    elif observation.step == 4:
        return 0  # Камень
    elif observation.step == 5:
        return 0  # Камень
    elif observation.step == 6:
        return 2  # Ножницы
    elif observation.step == 7:
        return 2  # Ножницы
    elif observation.step == 8:
        return 2  # Ножницы
    else:
         if 0 <= observation.step % 9 < 3:
             return 1
         elif 3 <= observation.step % 9 < 6:
             return 0
         else:
            return 2

Overwriting repeating_pattern_agent_bbbkkknnn.py


In [20]:
%%writefile rock_then_counter.py
def rock_then_counter(observation, configuration):
    if observation.step == 0:
        return 0
    elif observation.lastOpponentAction == 0:
        return 1
    elif observation.lastOpponentAction == 1:
        return 2
    elif observation.lastOpponentAction == 2:
        return 0

Overwriting rock_then_counter.py


In [21]:
agents = [
    "rock_agent.py",        # Агент, всегда выбирающий камень.
    "paper_agent.py",       # Агент, всегда выбирающий бумагу.
    "scissors_agent.py",    # Агент, всегда выбирающий ножницы.
    "cycle_agent.py",       # Агент, циклически меняющий выбор (камень, ножницы, бумага).
    "scissors_cycle_agent.py", # Агент, циклически меняющий выбор (ножницы, бумага, камень)
    "paper_cycle_agent.py", # Агент, циклически меняющий выбор (бумага, камень, ножницы)
    "pattern_agent_kknnbb.py", # Агент, использующий паттерн ККННББ.
    "repeating_pattern_agent_nnbbkk.py", # Агент, использующий повторяющийся паттерн ННББКК
    "repeating_pattern_agent_bbkknn.py", # Агент, использующий повторяющийся паттерн ББККНН
    "repeating_pattern_agent_kkknnnbbb.py", # Агент, использующий повторяющийся паттерн КККНННБББ
    "repeating_pattern_agent_nnnbbbkkk.py", # Агент, использующий повторяющийся паттерн НННБББККК
    "repeating_pattern_agent_bbbkkknnn.py", # Агент, использующий повторяющийся паттерн БББКККННН
    "rock_then_counter.py" # Агент, начинающий с камня, а затем пытающийся контрить ходы противника.
]

# Создаем словарь для подсчета очков.
# Ключи словаря - названия агентов (строки), значения - начальное количество очков (0).
scores = {agent: 0 for agent in agents}

# Количество игр между каждой парой агентов.
num_games = 20

# Проведение турнира.
# Внешний цикл перебирает всех агентов.
for i in range(len(agents)):
    # Внутренний цикл перебирает всех агентов, начиная со следующего после i,
    # чтобы избежать повторных игр (например, rock_agent vs paper_agent и paper_agent vs rock_agent).
    for j in range(i + 1, len(agents)):
        # Создаем турнир "камень-ножницы-бумага" с заданным количеством раундов.
        tournament = make('rps', configuration={'episodeSteps': num_games})
        # Запускаем турнир между агентами agents[i] и agents[j].
        results = tournament.run([agents[i], agents[j]])

        # Инициализируем счетчики очков для текущей пары агентов.
        score_i = 0
        score_j = 0

        # Выводим разделитель, чтобы отделить результаты игр между разными парами агентов.
        print(f"\n--- Игра между {agents[i]} и {agents[j]} ---")

        # Цикл по результатам каждой игры (каждого раунда).
        for k, result in enumerate(results):
            # Извлекаем действия (выборы) агентов в текущем раунде.
            action_i = result[0]['action']
            action_j = result[1]['action']
            # Извлекаем награды (результаты) для каждого агента в текущем раунде.
            reward_i = result[0]['reward']
            reward_j = result[1]['reward']

            # Выводим информацию о текущем раунде: номера раунда и выборы агентов.
            print(f"Раунд {k+1}: {agents[i]} выбрал {action_i}, {agents[j]} выбрал {action_j}. ", end="")

            # Определяем победителя раунда и выводим соответствующее сообщение.
            if reward_i > reward_j:
                score_i += 1 # Увеличиваем счетчик первого агента.
                print(f"{agents[i]} победил!")
            elif reward_i < reward_j:
                score_j += 1 # Увеличиваем счетчик второго агента.
                print(f"{agents[j]} победил!")
            else:
                print("Ничья!")

        # Добавляем набранные очки к общему счету агентов.
        scores[agents[i]] += score_i
        scores[agents[j]] += score_j

# Определение победителя турнира.
# Используем функцию max с ключом scores.get, чтобы найти агента с максимальным количеством очков.
winner = max(scores, key=scores.get)
# Выводим имя победителя и его количество очков.
print(f"\nПобедитель: Агент {winner} с {scores[winner]} очками.")


--- Игра между rock_agent.py и paper_agent.py ---
Раунд 1: rock_agent.py выбрал 0, paper_agent.py выбрал 0. Ничья!
Раунд 2: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 3: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 4: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 5: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 6: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 7: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 8: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 9: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 10: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 11: rock_agent.py выбрал 0, paper_agent.py выбрал 1. paper_agent.py победил!
Раунд 12: rock_agent.py выбрал 0, paper_agent.py выбрал 1. pap